# CNN

In [1]:
import torch
import torch.nn as nn
import torchvision.transforms as transforms
import torchvision.datasets as dsets
from torch.autograd import Variable
import matplotlib.pyplot as plt
import numpy as np
%matplotlib inline

In [2]:
train_dataset = dsets.MNIST(root = './root',
                            train = True , 
                            transform = transforms.ToTensor(),
                            download = True)

test_dataset = dsets.MNIST(root = './data',
                          train = False,
                          transform = transforms.ToTensor(),
                          download = True)

In [3]:
bs = 100
n_iters = 6000
num_epochs = n_iters/(len(train_dataset)/bs)
num_epochs = int(num_epochs)

In [4]:
train_loader = torch.utils.data.DataLoader(dataset = train_dataset,
                                          batch_size = bs, 
                                          shuffle = True)

test_loader = torch.utils.data.DataLoader(dataset = test_dataset,
                                          batch_size = bs, 
                                          shuffle = False)

In [5]:
class CNNModel(nn.Module):
    
    def __init__(self):
        
        super(CNNModel,self).__init__()
        
        self.cnn1 = nn.Conv2d(in_channels = 1, out_channels = 16, kernel_size = 5, stride = 1, padding = 2)
        self.relu1 = nn.ReLU()
        
        self.maxpool1 = nn.MaxPool2d(kernel_size = 2)
        
        self.cnn2 = nn.Conv2d(in_channels = 16, out_channels = 32, kernel_size = 5, stride = 1, padding = 2)
        self.relu2 = nn.ReLU()
        
        self.maxpool2 = nn.MaxPool2d(kernel_size = 2)
        
        self.fc1 = nn.Linear(32*7*7,10)
        
    def forward(self,x):
        
        out = self.cnn1(x)
        out = self.relu1(out)
        out = self.maxpool1(out)
        
        out = self.cnn2(out)
        out = self.relu2(out)
        out = self.maxpool2(out)
        
        out = out.view(out.size(0),-1)
        
        out = self.fc1(out)
        
        return out
        

In [6]:
model = CNNModel()
model.cuda()

CNNModel(
  (cnn1): Conv2d(1, 16, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
  (relu1): ReLU()
  (maxpool1): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (cnn2): Conv2d(16, 32, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
  (relu2): ReLU()
  (maxpool2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (fc1): Linear(in_features=1568, out_features=10, bias=True)
)

In [7]:
learning_rate = 0.01
optimizer = torch.optim.SGD(model.parameters(),lr = learning_rate)
criterion = nn.CrossEntropyLoss()

In [9]:
for epoch in range(int(num_epochs)):
    
    for i,(images,labels) in enumerate(train_loader):
        
        images = Variable(images.cuda())
        labels = Variable(labels.cuda())
        
        optimizer.zero_grad()
        
        outputs = model(images)
        
        loss = criterion(outputs,labels)
        
        loss.backward()
        
        optimizer.step()
        
    correct = 0
    total = 0
    
    for images,labels in train_loader:
        
        images = Variable(images.cuda())
        
        outputs = model(images.cuda())
        
        _, pred = torch.max(outputs.data,1)
        
        total += labels.size(0)
        
        correct += (pred.cpu() == labels.cpu()).sum()
        
    accuracy = 100*correct/total
    print("{} . loss : {} ,accuracy : {}".format(epoch+1,loss.data[0],accuracy.item()))
    

C:\Users\Abhishek\Anaconda3\envs\fastai\lib\site-packages\ipykernel_launcher.py:34: UserWarning: invalid index of a 0-dim tensor. This will be an error in PyTorch 0.5. Use tensor.item() to convert a 0-dim tensor to a Python number


1 . loss : 0.3626522123813629 ,accuracy : 90
2 . loss : 0.24135816097259521 ,accuracy : 93
3 . loss : 0.20865169167518616 ,accuracy : 94
4 . loss : 0.23470892012119293 ,accuracy : 95
5 . loss : 0.17341026663780212 ,accuracy : 96
6 . loss : 0.06411638855934143 ,accuracy : 96
7 . loss : 0.09124908596277237 ,accuracy : 97
8 . loss : 0.09098806977272034 ,accuracy : 97
9 . loss : 0.021289082244038582 ,accuracy : 97
10 . loss : 0.048097334802150726 ,accuracy : 97
